# Problem 3 Sample Code

This sample code is meant as a guide on how to use PyTorch and how to use the relevant model layers. This not a guide on how to design a network and the network in this example is intentionally designed to have poor performace.

In [134]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset, Dataset, DataLoader

In [135]:
df = pd.read_csv('out.csv')
df.drop(['uid'], axis=1, inplace=True)

train = df.iloc[0:15000 ]
test = df.iloc[15000:16080]

x_train = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
y_train = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.float32)

x_test = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
y_test = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.float32)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

We can convert images to numpy arrays and plot them with matplotlib:

## Network Definition
Let's instantiate a model and take a look at the layers.

In [136]:
model = nn.Sequential(
    # In problem 2, we don't use the 2D structure of an image at all. Our network
    # takes in a flat vector of the pixel values as input.

    nn.Flatten(),
    nn.Linear(4, 3),
    nn.Sigmoid(),
    nn.Linear(3, 1),
    nn.Softmax(dim=1)
)
print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=4, out_features=3, bias=True)
  (2): Sigmoid()
  (3): Linear(in_features=3, out_features=1, bias=True)
  (4): Softmax(dim=1)
)


## Training
We also choose an optimizer and a loss function.

In [137]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = nn.BCEWithLogitsLoss()

We could write our training procedure manually and directly index the `Dataset` objects, but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:

In [138]:
train_loader = DataLoader(train_dataset, batch_size=40, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=40, shuffle=True) 

torch.manual_seed(15552494823729223621)

We now write our backpropagation loop, training for 10 epochs.

In [139]:
# Some layers, such as Dropout, behave differently during training
model.train()

for epoch in range(20):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        
        target = target.unsqueeze(1)
        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()


    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: nan
Train Epoch: 2  Loss: nan
Train Epoch: 3  Loss: nan
Train Epoch: 4  Loss: nan
Train Epoch: 5  Loss: nan
Train Epoch: 6  Loss: nan
Train Epoch: 7  Loss: nan
Train Epoch: 8  Loss: nan
Train Epoch: 9  Loss: nan
Train Epoch: 10  Loss: nan
Train Epoch: 11  Loss: nan
Train Epoch: 12  Loss: nan
Train Epoch: 13  Loss: nan
Train Epoch: 14  Loss: nan
Train Epoch: 15  Loss: nan
Train Epoch: 16  Loss: nan


## Testing
We can perform forward passes through the network without saving gradients.

In [ ]:
# Putting layers like Dropout into evaluation mode
model.eval()

test_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        target = target.unsqueeze(1)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

Test set: Average loss: nan, Accuracy: 7619/15000 (50.7933)
